<a href="https://colab.research.google.com/github/dhwan-dj/S4pjt_NLP_koBERT/blob/main/kobert_sentiment_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 개요 : KoBERT를 이용하여 한국어 문장을 여러 클래스로 분류하는 모델을 만드는 것인데, 공포, 놀람, 분노, 슬픔, 중립, 행복, 혐오와 같은 감정이 느껴지는 짧은 대화 텍스트를 각각 어떠한 감정의 텍스트인지 분류하는 모델을 만드는 것이다. 예를 들어 "앗 깜작이야!" 라는 문장을 입력하면 '놀람'이라는 클래스로 예측을 하도록 학습시키는 것
- 코드는 KoBERT 깃허브에 있는 네이버 영화평 이중분류 예시 코드를 바탕으로 작성
- 참고 블로그 (https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab)

# 1. Colab 환경설정

- 라이브러리, 모듈 설치, koBERT 모델 불러오기
- 예시 코드의 Dataset = '한국어 감정 정보가 포함된 단발성 대화'
- 본 코드의 Dataset : 감성대화말뭉치(https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86)

<세팅 및 파라미터>  

    Python >= 3.6
    PyTorch >= 1.70
    Transformers = 3.0.2
    Colab
    batch size = 64
    epochs = 10

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=689011 sha256=4563a958248f4636d2ba431f4c069cee919d46f5262263bd328edea6d5dfe85e
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [29]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=540937cc4dad04fd882e8e767dfd034e896011d827328ac47833096deb9672d6
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [2]:
# 깃허브에서 KoBERT 파일들 다운로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-7e83vfxa
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-7e83vfxa
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf 의 kobert_tokenizer 폴더를 다운받는 코드.
#!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [4]:
import torch             # pytorch framework
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
#kobert
from kobert.utils import get_tokenizer
# https://github.com/SKTBrain/KoBERT/tree/master/kobert/utils 의 utils.py 파일에서 get_tokenizer 메서드를 불러오는 코드
from kobert.pytorch_kobert import get_pytorch_kobert_model
# https://github.com/SKTBrain/KoBERT/tree/master/kobert 의 pytorch_kobert.py 파일에서 get_pytorch_kobert_model 메서드를 불러오는 코드

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
#GPU 사용
device = torch.device("cuda:0")

In [7]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


- BERT는 이미 누군가가 학습해둔 모델을 사용한다(pre-trained model)는 것을 뜻한다. 따라서 사용하는 model과 tokenizer는 항상 mapping 관계여야 한다. 예를 들어서 U 팀이 개발한 BERT를 사용하는데, V팀이 개발한 BERT의 tokenizer를 사용하면 model은 텍스트를 이해할 수 없다. U팀의 BERT의 토크나이저는 '우리'라는 단어를 23번으로 int encoding하는 반면에, V라는 BERT의 tokenizer는 '우리'라는 단어를 103번으로 int encoding해 단어와 mapping 되는 정보 자체가 달라지기 때문이다. 
(https://hoit1302.tistory.com/159)

# 2. 데이터셋 로드, 전처리

In [8]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
train = pd.read_excel('/content/drive/MyDrive/data/sentiment_train.xlsx')
val = pd.read_excel('/content/drive/MyDrive/data/sentiment_val.xlsx')

In [10]:
df = pd.concat([train,val])

In [11]:
len(df) # 문장 = 58,271개

58271

In [12]:
df.sample(5)

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
1517,1518,청소년,여성,가족관계,해당없음,당황,남의 시선을 의식하는,우리 오빠는 집 안에서는 게임만 하는데 밖에서는 뭐든 열심히 하는 척해서 당황했어.,오빠의 이중적인 태도 때문에 놀라신 것 같네요.,오빠의 실체를 남들도 알아야 한다고 생각해.,오빠의 실체를 남들이 알았으면 좋겠군요. 이런 상황에서 사용자님이 할 수 있는 행동...,오빠에게 먼저 내 생각을 말해 봐야겠어.,오빠와 대화가 우선이라고 생각하시는군요.
3544,3545,노년,남성,대인관계,만성질환 무,기쁨,느긋,자식들이 하고 싶은 공부가 있다고 해서 내가 열심히 도와줄 거라고 말했어. 아주 뿌듯해.,자제 분들이 하고 싶은 공부를 응원하셨군요. 많이 뿌듯하셨겠어요.,아주 뿌듯했지. 자식들의 기쁜 표정이 아직도 선해.,지금의 그 기분을 간직할 수 있는 방법은 무엇이 있을까요?,오늘 이 기분을 일기로 써야겠어.,좋은 추억이 기록을 통해 오래 간직되었으면 좋겠어요.
5779,5780,청년,남성,대인관계,해당없음,슬픔,마비된,이번에 학교생활을 하다가 졸업을 했는데 내 유일한 친구들이 다 대학교 친구들이라 졸...,많이 외로우셨겠습니다. 혹시 현재는 친구들과 서로 연락을 하고 계시지 않으신가요?,아무래도 졸업을 하고 취업을 해야 하다 보니 이전만큼 연락을 잘할 수 없는 것 같아.,친구분들도 철수 님을 많이 보고 싶어 할 것 같네요.,NaN,NaN
46097,46098,중년,여성,"직장, 업무 스트레스",해당없음,불안,조심스러운,일을 처음엔 좋아서 시작했는데 사람들이 난 틱 장애가 있다고 일에서 배척해서 너무 슬퍼.,틱 장애라는 이유로 직장 사람들이 배척해서 스트레스가 많으셨군요.,난 사람이랑 대화하는 거 좋아하는데 이젠 흥미도 사라지고 있어. 직장 그만둘까?,이 상황에서 벗어나기 위해 무엇을 할 수 있을까요?,좋아하는 일을 직장 사람들 때문에 그만두지 않을 거야! 일에서 배척될 때마다 맞서겠어!,이 상황을 사람들에게 맞서면서 잘 극복해 내시길 응원할게요!
15104,15105,청년,남성,"연애,결혼,출산",해당없음,분노,악의적인,요즘 여자친구 연락이 귀찮더라. 점심시간에 전화가 왔는데 그냥 바쁘다고 끊어 버렸어.,여자친구와 연락하는 일이 귀찮으셨군요.,답장해야한다는 의무감을 느껴서 그런가. 유난히 여자친구 연락이 귀찮더라고.,무언가 의무감을 덜 수 있는 방법은 없을까요?,여자친구와 자잘한 연락 대신 자기 전에 길게 통화하는 건 어떤지 물어봐야겠다.,이야기가 잘 마무리 되어 두 분 다 기쁜 마음으로 연락하실 수 있었으면 좋겠어요.


- 6번째 칼럼 '감정_대분류'를 라벨 삼아 입력문장의 감정을 예측하기로 한다.

In [13]:
df.감정_대분류.value_counts()

불안    10433
분노    10417
상처    10150
슬픔    10128
당황     9804
기쁨     7339
Name: 감정_대분류, dtype: int64

In [14]:
# 주관적으로 긍정에서 부정으로 숫자 라벨을 매겨보았다.
df.loc[(df['감정_대분류'] == "기쁨"), 'sentiment'] = '0'    #기쁨 => 0
df.loc[(df['감정_대분류'] == "당황"), 'sentiment'] = '1'    #당황 => 1
df.loc[(df['감정_대분류'] == "불안"), 'sentiment'] = '2'    #불안 => 2
df.loc[(df['감정_대분류'] == "분노"), 'sentiment'] = '3'    #분노 => 3
df.loc[(df['감정_대분류'] == "슬픔"), 'sentiment'] = '4'    #슬픔 => 4
df.loc[(df['감정_대분류'] == "상처"), 'sentiment'] = '5'    #상처 => 5

In [15]:
# 문장과 문장의 감정을 리스트로 저장
data_list = []
for q, label in zip(df['사람문장1'], df['sentiment']):
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [16]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['일은 왜 해도 해도 끝이 없을까? 화가 난다.', '3']
['최근에 업무가 너무 많이 늘어난 것 같아 힘들어.', '3']
['친구들은 다 취업에 성공했는데 나만 못한 것 같아.', '2']
['친구에게 내가 간암에 걸려서 술을 마실 수 없다고 하자 거짓말하지 말라고 했어.', '5']
['의사가 분명히 수술이 잘 되었다고 했거든. 그런데 삼 주가 되도록 몸을 움직일 수 없어.', '5']
['이번에 주식을 샀는데 주가가 너무 떨어져 손해가 이만저만이 아냐.', '4']
['친구들 모두 결혼하고 나만 혼자 남아서 쓸쓸하네.', '1']


# 3. Train과 Test Set 준비

In [17]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.15, random_state=0)

In [18]:
print(len(dataset_train))
print(len(dataset_test))

49530
8741


# 4. koBERT의 입력 데이터 형태로 만들기
- 모델의 입력으로 활용하기 위해 토큰화, 라벨과 문장의 인덱스 인코딩, 패딩 등이 필요하다.
- 예시 코드의 입력데이터 형태로 바꿔주는 클래스 활용

In [19]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
      transform = nlp.data.BERTSentenceTransform(
          bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)   # BERTSentenceTransform 으로 토큰화, 패딩
      
      self.sentences = [transform([i[sent_idx]]) for i in dataset]
      self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
      return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
      return (len(self.labels))

In [20]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [21]:
# BERTTokenizer와 위의 Class로 Tokenize, Padding
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


- 토큰화와 패딩이 잘 이루어졌을까?
- 데이터는 3개의 array인데, 첫번째는 패딩된 시퀀스, 두번째는 길이와 타입, 세번째는 어텐션 마스크 시퀀스라고 한다.
- BERT에 데이터가 입력되었을 때 어텐션 함수가 적용되어 연산이 된다. 이때 1로 패딩된 값들은 연산할 필요가 없기 때문에 연산을 하지 않아도 된다고 알려주는 데이터가 있어야 하는데 그게 바로 어텐션 마스크 시퀀스 라고 한다. (아직 이해x)

In [22]:
data_train[0]

(array([   2, 1261, 7318, 6941, 7096, 3869,  784, 5579,  889, 6122, 6198,
        4688, 7126, 7120, 4304, 4213, 6896, 1934, 4627, 6897,  784, 5573,
         517, 6976, 6797, 6323, 6072, 6855,  517,   54,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(31, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [23]:
# torch 형식의 dataset을 만들어준다.
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# 5. koBERT 학습모델 만들기
- num_classes = 6 (6개의 감정 클래스)

In [24]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 6,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [25]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정(linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy() / max_indices.size()[0]
    return train_acc
    
train_dataloader

# 6. BERT model 학습시키기

In [26]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-26-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/774 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.763039469718933 train acc 0.171875
epoch 1 batch id 201 loss 1.417669415473938 train acc 0.28194962686567165
epoch 1 batch id 401 loss 1.2790253162384033 train acc 0.39643079800498754
epoch 1 batch id 601 loss 1.1170663833618164 train acc 0.44693739600665555
epoch 1 train acc 0.47223266394903324


<ipython-input-26-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/137 [00:00<?, ?it/s]

...

epoch 4 test acc 0.5961481554547248


  0%|          | 0/774 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 1.1079446077346802 train acc 0.625
epoch 5 batch id 201 loss 0.5921837091445923 train acc 0.7380286069651741
epoch 5 batch id 401 loss 0.5954712629318237 train acc 0.7449735037406484
epoch 5 batch id 601 loss 0.6518911719322205 train acc 0.7518198835274542
epoch 5 train acc 0.7511416287980041


  0%|          | 0/137 [00:00<?, ?it/s]

epoch 5 test acc 0.5934941556520024


- Epochs 5 학습은 약 47분이 걸렸고, train dataset에 대해서는 0.747, test dataset에 대해서는 0.594의 정확도를 기록했다.

- 이거 좀 낮은데;;

- 참고블로그에서는 정확도가 높았는데, 이에 대한 설명 :
train dataset에 대해서는 0.979, test dataset에 대해서는 0.918의 정확도를 기록
이렇게 높은 정확도를 기록하는 이유는 바로 데이터셋에 있다. 진행하고 있는 프로젝트에서 해당 프로젝트의 성격에 맞게 3만 7천 여개 정도의 데이터를 직접 재분류하고 있는데, 사람이 생각하기에는 분명히 같은 내용으로 인지되는 문장이지만 사소한 단어를 한 두개 없애서, 어떨 때는 중요한 단어를 제거해서, 짧게 잘라서, 길게 늘여뜨려서, 문장 부호를 다르게, 감탄사를 추가해서 등등 중복인 듯 중복 아닌 데이터로 학습시켰기 때문에 이렇게 높은 정확도가 나온 것이라 판단된다.
(https://hoit1302.tistory.com/159)

In [33]:
from keras.callbacks import Callback
from seqeval.metrics import f1_score, classification_report

In [34]:
class F1score(Callback):
    def __init__(self, value = 0.0, use_char=True):
        super(F1score, self).__init__()
        self.value = value
        self.use_char = use_char

    def sequences_to_tags(self, sequences): # 예측값을 index_to_ner를 사용하여 태깅 정보로 변경하는 함수.
      result = []
      for sequence in sequences: # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
          tag = []
          for pred in sequence: # 시퀀스로부터 예측값을 하나씩 꺼낸다.
              pred_index = np.argmax(pred) # 예를 들어 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
              tag.append(index_to_ner[pred_index].replace("PAD", "O")) # 'PAD'는 'O'로 변경
          result.append(tag)
      return result

    # 에포크가 끝날 때마다 실행되는 함수
    def on_epoch_end(self, epoch, logs={}):

      # char Embedding을 사용하는 경우
      if self.use_char:
        X_test = self.validation_data[0]
        X_char_test = self.validation_data[1]
        y_test = self.validation_data[2]
        y_predicted = self.model.predict([X_test, X_char_test])

      else:
        X_test = self.validation_data[0]
        y_test = self.validation_data[1]
        y_predicted = self.model.predict([X_test])

      pred_tags = self.sequences_to_tags(y_predicted)
      test_tags = self.sequences_to_tags(y_test)

      score = f1_score(pred_tags, test_tags)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(test_tags, pred_tags))

      # F1-score가 지금까지 중 가장 높은 경우
      if score > self.value:
        print('f1_score improved from %f to %f, saving model to best_model.h5'%(self.value, score))
        self.model.save('best_model.h5')
        self.value = score
      else:
        print('f1_score did not improve from %f'%(self.value))

In [35]:
f1score = F1score(use_char=True)

y_predicted = model.predict([X_test, X_char_test])
pred_tags = f1score.sequences_to_tags(y_predicted)
test_tags = f1score.sequences_to_tags(y_test)

print(classification_report(test_tags, pred_tags))

NameError: ignored

In [ ]:
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

In [31]:
from typing import Tuple

import torch


class F1Score:
    """
    Class for f1 calculation in Pytorch.
    """

    def __init__(self, average: str = 'weighted'):
        """
        Init.

        Args:
            average: averaging method
        """
        self.average = average
        if average not in [None, 'micro', 'macro', 'weighted']:
            raise ValueError('Wrong value of average parameter')

    @staticmethod
    def calc_f1_micro(predictions: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """
        Calculate f1 micro.

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels

        Returns:
            f1 score
        """
        true_positive = torch.eq(labels, predictions).sum().float()
        f1_score = torch.div(true_positive, len(labels))
        return f1_score

    @staticmethod
    def calc_f1_count_for_label(predictions: torch.Tensor,
                                labels: torch.Tensor, label_id: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Calculate f1 and true count for the label

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels
            label_id: id of current label

        Returns:
            f1 score and true count for label
        """
        # label count
        true_count = torch.eq(labels, label_id).sum()

        # true positives: labels equal to prediction and to label_id
        true_positive = torch.logical_and(torch.eq(labels, predictions),
                                          torch.eq(labels, label_id)).sum().float()
        # precision for label
        precision = torch.div(true_positive, torch.eq(predictions, label_id).sum().float())
        # replace nan values with 0
        precision = torch.where(torch.isnan(precision),
                                torch.zeros_like(precision).type_as(true_positive),
                                precision)

        # recall for label
        recall = torch.div(true_positive, true_count)
        # f1
        f1 = 2 * precision * recall / (precision + recall)
        # replace nan values with 0
        f1 = torch.where(torch.isnan(f1), torch.zeros_like(f1).type_as(true_positive), f1)
        return f1, true_count

    def __call__(self, predictions: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
        """
        Calculate f1 score based on averaging method defined in init.

        Args:
            predictions: tensor with predictions
            labels: tensor with original labels

        Returns:
            f1 score
        """

        # simpler calculation for micro
        if self.average == 'micro':
            return self.calc_f1_micro(predictions, labels)

        f1_score = 0
        for label_id in range(1, len(labels.unique()) + 1):
            f1, true_count = self.calc_f1_count_for_label(predictions, labels, label_id)

            if self.average == 'weighted':
                f1_score += f1 * true_count
            elif self.average == 'macro':
                f1_score += f1

        if self.average == 'weighted':
            f1_score = torch.div(f1_score, len(labels))
        elif self.average == 'macro':
            f1_score = torch.div(f1_score, len(labels.unique()))

        return f1_score

In [32]:
from sklearn.metrics import f1_score
import numpy as np
errors = 0
for _ in range(10):
    labels = torch.randint(1, 10, (4096, 100)).flatten()
    predictions = torch.randint(1, 10, (4096, 100)).flatten()
    labels1 = labels.numpy()
    predictions1 = predictions.numpy()

    for av in ['micro', 'macro', 'weighted']:
        f1_metric = F1Score(av)
        my_pred = f1_metric(predictions, labels)
        
        f1_pred = f1_score(labels1, predictions1, average=av)
        
        if not np.isclose(my_pred.item(), f1_pred.item()):
            print('!' * 50)
            print(f1_pred, my_pred, av)
            errors += 1

if errors == 0:
    print('No errors!')

No errors!


# 7. 새로운 문장 테스트
- 새로운 문장을 koBERT의 입력 형식으로 바꿔주는 predict 함수를 정의한다.

In [27]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("당황이")
            elif np.argmax(logits) == 2:
                test_eval.append("불안이")
            elif np.argmax(logits) == 3:
                test_eval.append("분노가")
            elif np.argmax(logits) == 4:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 5:
                test_eval.append("상처가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [28]:
# 질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 올해 부동산 가격이 어떻게 될지 궁금하네
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : 이번에 터키에 강진이 발생해서 사상자가 많대.
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 학력 위조는 왜 하는걸까
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 미세먼지가 심한 날이야
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : 저녁은 지옥에서 먹는 걸로 하자구.
>> 입력하신 내용에서 상처가 느껴집니다.


하고싶은 말을 입력해주세요 : 신혼여행은 하와이로 가기로 했어
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 차 브레이크 작동이 안돼!
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : 주식이 반토막 났어..
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 귀신이 나오는 폐가가 근처에 있대.
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : 이러다가는 다 죽어!
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 어떻게 하면 니가 공포를 감지하니?
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : 0


# **Result**
- 학습데이터 정확도
- 테스트데이터 정확도
- F1, Precision, Recall 점수를 계산
- 새로운 문장에 대한 정확도

- 한계점

- 개선점

- 기대효과